# Builder (Polyglot)

In [ ]:
#!import ../../lib/fsharp/Notebooks.dib
#!import ../../lib/fsharp/Testing.dib

In [ ]:
#r @"../../../../../../../.nuget/packages/fsharp.control.asyncseq/3.2.1/lib/netstandard2.1/FSharp.Control.AsyncSeq.dll"
#r @"../../../../../../../.nuget/packages/system.reactive/6.0.1-preview.1/lib/net6.0/System.Reactive.dll"
#r @"../../../../../../../.nuget/packages/system.reactive.linq/6.0.1-preview.1/lib/netstandard2.0/System.Reactive.Linq.dll"
#r @"../../../../../../../.nuget/packages/argu/6.1.5/lib/netstandard2.0/Argu.dll"
#r @"../../../../../../../.nuget/packages/system.commandline/2.0.0-beta4.22272.1/lib/net6.0/System.CommandLine.dll"

In [ ]:
#!import ../../lib/fsharp/Common.fs
#!import ../../lib/fsharp/CommonFSharp.fs
#!import ../../lib/fsharp/Async.fs
#!import ../../lib/fsharp/AsyncSeq.fs
#!import ../../lib/fsharp/Networking.fs
#!import ../../lib/fsharp/Runtime.fs
#!import ../../lib/fsharp/FileSystem.fs

In [ ]:
open Common
open FileSystem.Operators

## buildProject

In [ ]:
let inline buildProject runtime outputDir path = async {
    let fullPath = path |> System.IO.Path.GetFullPath
    let fileDir = fullPath |> System.IO.Path.GetDirectoryName
    let extension = fullPath |> System.IO.Path.GetExtension

    let getLocals () = $"fullPath: {fullPath} / {getLocals ()}"
    trace Debug (fun () -> "buildProject") getLocals

    match extension with
    | ".fsproj" -> ()
    | _ -> failwith "Invalid project file"

    let runtimes =
        runtime
        |> Option.map List.singleton
        |> Option.defaultValue [ "linux-x64"; "win-x64" ]
    
    let outputDir = outputDir |> Option.defaultValue "dist"

    return!
        runtimes
        |> List.map (fun runtime -> async {
            let! exitCode, _result =
                Runtime.executeWithOptionsAsync
                    {
                        Command = $@"dotnet publish ""{path}"" --configuration Release --output ""{outputDir}"" --runtime {runtime}"
                        CancellationToken = None
                        OnLine = None
                        WorkingDirectory = Some fileDir
                    }

            return exitCode
        })
        |> Async.Sequential
        |> Async.map Array.sum
}

## persistCodeProject

In [ ]:
let inline persistCodeProject packages modules name code = async {
    let getLocals () = $"packages: {packages} / modules: {modules} / name: {name} / code.Length: {code |> String.length} / {getLocals ()}"
    trace Debug (fun () -> "persistCodeProject") getLocals

    let repositoryRoot = FileSystem.getSourceDirectory () |> FileSystem.findParent ".paket" false

    let targetDir = repositoryRoot </> "target/polyglot/builder" </> name
    System.IO.Directory.CreateDirectory targetDir |> ignore

    let filePath = targetDir </> $"{name}.fs" |> System.IO.Path.GetFullPath
    do! code |> FileSystem.writeAllTextExists filePath

    let modulesCode =
        modules
        |> List.map (fun path -> $"""<Compile Include="{repositoryRoot </> path}" />""")
        |> String.concat "\n        "

    let fsprojPath = targetDir </> $"{name}.fsproj"
    let fsprojCode = $"""<Project Sdk="Microsoft.NET.Sdk">
    <PropertyGroup>
        <TargetFramework>net8.0</TargetFramework>
        <LangVersion>preview</LangVersion>
        <RollForward>Major</RollForward>
        <TargetLatestRuntimePatch>true</TargetLatestRuntimePatch>
        <PublishAot>false</PublishAot>
        <PublishTrimmed>false</PublishTrimmed>
        <PublishSingleFile>true</PublishSingleFile>
        <SelfContained>true</SelfContained>
        <Version>0.0.1-alpha.1</Version>
        <OutputType>Exe</OutputType>
    </PropertyGroup>

    <ItemGroup>
        {modulesCode}
        <Compile Include="{filePath}" />
    </ItemGroup>

    <Import Project="{repositoryRoot}/.paket/Paket.Restore.targets" />
</Project>
"""
    do! fsprojCode |> FileSystem.writeAllTextExists fsprojPath

    let paketReferencesPath = targetDir </> "paket.references"
    let paketReferencesCode =
        "FSharp.Core" :: packages
        |> String.concat "\n"
    do! paketReferencesCode |> FileSystem.writeAllTextExists paketReferencesPath

    return fsprojPath
}

## buildCode

In [ ]:
let inline buildCode runtime packages modules outputDir name code = async {
    let! fsprojPath = code |> persistCodeProject packages modules name
    return! fsprojPath |> buildProject runtime outputDir
}

In [ ]:
//// test

"1 + 1"
|> buildCode None [] [] None "test1"
|> Async.runWithTimeout 60000
|> _equal (Some 0)

00:00:00 #1 [Debug] persistCodeProject / packages: [] / modules: [] / name: test1 / code.Length: 5
00:00:00 #2 [Debug] buildProject / fullPath: /home/runner/work/polyglot/polyglot/target/polyglot/builder/test1/test1.fsproj
00:00:00 #3 [Debug] executeAsync / options: { Command =
   "dotnet publish "/home/runner/work/polyglot/polyglot/target/polyglot/builder/test1/test1.fsproj" --configuration Release --output "dist" --runtime linux-x64"
  WorkingDirectory =
   Some "/home/runner/work/polyglot/polyglot/target/polyglot/builder/test1"
  CancellationToken = None
  OnLine = None }
00:00:00 #4 [Verbose] > MSBuild version 17.9.4+90725d08d for .NET
00:00:00 #5 [Verbose] >   Determining projects to restore...
00:00:02 #6 [Verbose] >   Restored /home/runner/work/polyglot/polyglot/target/polyglot/builder/test1/test1.fsproj (in 1.8 sec).
00:00:03 #7 [Verbose] > /home/runner/work/polyglot/polyglot/target/polyglot/builder/test1/test1.fs(1,1): warning FS0020: The result of this expression has type 'in

In [ ]:
//// test

"1 + a"
|> buildCode None [] [] None "test2"
|> Async.runWithTimeout 60000
|> _equal (Some 2)

00:00:09 #21 [Debug] persistCodeProject / packages: [] / modules: [] / name: test2 / code.Length: 5
00:00:09 #22 [Debug] buildProject / fullPath: /home/runner/work/polyglot/polyglot/target/polyglot/builder/test2/test2.fsproj
00:00:09 #23 [Debug] executeAsync / options: { Command =
   "dotnet publish "/home/runner/work/polyglot/polyglot/target/polyglot/builder/test2/test2.fsproj" --configuration Release --output "dist" --runtime linux-x64"
  WorkingDirectory =
   Some "/home/runner/work/polyglot/polyglot/target/polyglot/builder/test2"
  CancellationToken = None
  OnLine = None }
00:00:09 #24 [Verbose] > MSBuild version 17.9.4+90725d08d for .NET
00:00:10 #25 [Verbose] >   Determining projects to restore...
00:00:10 #26 [Verbose] >   Restored /home/runner/work/polyglot/polyglot/target/polyglot/builder/test2/test2.fsproj (in 252 ms).
00:00:12 #27 [Verbose] > /home/runner/work/polyglot/polyglot/target/polyglot/builder/test2/test2.fs(1,5): error FS0039: The value or constructor 'a' is not de

## readFile

In [ ]:
let inline readFile path = async {
    let! code = path |> FileSystem.readAllTextAsync

    let code = System.Text.RegularExpressions.Regex.Replace (
        code,
        @"( *)(let\s+main\s+.*?\s*=)",
        fun m -> m.Groups.[1].Value + "[<EntryPoint>]\n" + m.Groups.[1].Value + m.Groups.[2].Value
    )

    let codeTrim = code |> String.trimEnd [||]
    return
        if codeTrim |> String.endsWith "\n()"
        then codeTrim |> String.substring 0 ((codeTrim |> String.length) - 2)
        else code
}

## buildFile

In [ ]:
let inline buildFile runtime packages modules path = async {
    let fullPath = path |> System.IO.Path.GetFullPath
    let dir = fullPath |> System.IO.Path.GetDirectoryName
    let name = fullPath |> System.IO.Path.GetFileNameWithoutExtension
    let! code = fullPath |> readFile
    return! code |> buildCode runtime packages modules (dir </> "dist" |> Some) name
}

## persistFile

In [ ]:
let inline persistFile packages modules path = async {
    let fullPath = path |> System.IO.Path.GetFullPath
    let name = fullPath |> System.IO.Path.GetFileNameWithoutExtension
    let! code = fullPath |> readFile
    return! code |> persistCodeProject packages modules name
}

## Arguments

In [ ]:
[<RequireQualifiedAccess>]
type Arguments =
    | [<Argu.ArguAttributes.MainCommand; Argu.ArguAttributes.ExactlyOnce>] Path of path : string
    | [<Argu.ArguAttributes.Unique>] Packages of packages : string list
    | [<Argu.ArguAttributes.Unique>] Modules of modules : string list
    | [<Argu.ArguAttributes.Unique>] Runtime of runtime : string
    | [<Argu.ArguAttributes.Unique>] Persist_Only

    interface Argu.IArgParserTemplate with
        member s.Usage =
            match s with
            | Path _ -> nameof Path
            | Packages _ -> nameof Packages
            | Modules _ -> nameof Modules
            | Runtime _ -> nameof Runtime
            | Persist_Only -> nameof Persist_Only

In [ ]:
//// test

Argu.ArgumentParser.Create<Arguments>().PrintUsage ()

USAGE: dotnet [--help] [--packages [<packages>...]] [--modules [<modules>...]]
              [--runtime <runtime>] [--persist-only] <path>

PATH:

    <path>                Path

OPTIONS:

    --packages [<packages>...]
                          Packages
    --modules [<modules>...]
                          Modules
    --runtime <runtime>   Runtime
    --persist-only        Persist_Only
    --help                display this list of options.


## main

In [ ]:
let main args =
    let argsMap = args |> Runtime.parseArgsMap<Arguments>

    let path =
        match argsMap.[nameof Arguments.Path] with
        | [ Arguments.Path path ] -> Some path
        | _ -> None
        |> Option.get

    let packages =
        match argsMap |> Map.tryFind (nameof Arguments.Packages) with
        | Some [ Arguments.Packages packages ] -> packages
        | _ -> []

    let modules =
        match argsMap |> Map.tryFind (nameof Arguments.Modules) with
        | Some [ Arguments.Modules modules ] -> modules
        | _ -> []

    let runtime =
        match argsMap |> Map.tryFind (nameof Arguments.Runtime) with
        | Some [ Arguments.Runtime runtime ] -> Some runtime
        | _ -> None

    let persistOnly = argsMap |> Map.containsKey (nameof Arguments.Persist_Only)

    if persistOnly
    then path |> persistFile packages modules |> Async.map (fun _ -> 0)
    else path |> buildFile runtime packages modules
    |> Async.runWithTimeout 60000
    |> function
        | Some exitCode -> exitCode
        | None -> 1

In [ ]:
//// test

let args =
    System.Environment.GetEnvironmentVariable "ARGS"
    |> Runtime.splitArgs
    |> Seq.toArray

match args with
| [||] -> 0
| args -> if main args = 0 then 0 else failwith "main failed"

0

00:00:15 #35 [Debug] persistCodeProject / packages: [Argu; FSharp.Control.AsyncSeq; System.CommandLine; ... ] / modules: [lib/fsharp/Common.fs; lib/fsharp/CommonFSharp.fs; lib/fsharp/Async.fs; ... ] / name: Builder / code.Length: 7079
00:00:15 #36 [Debug] buildProject / fullPath: /home/runner/work/polyglot/polyglot/target/polyglot/builder/Builder/Builder.fsproj
00:00:15 #37 [Debug] executeAsync / options: { Command =
   "dotnet publish "/home/runner/work/polyglot/polyglot/target/polyglot/builder/Builder/Builder.fsproj" --configuration Release --output "/home/runner/work/polyglot/polyglot/apps/builder/dist" --runtime linux-x64"
  WorkingDirectory =
   Some "/home/runner/work/polyglot/polyglot/target/polyglot/builder/Builder"
  CancellationToken = None
  OnLine = None }
00:00:15 #38 [Verbose] > MSBuild version 17.9.4+90725d08d for .NET
00:00:15 #39 [Verbose] >   Determining projects to restore...
00:00:16 #40 [Verbose] >   Paket version 8.0.0-alpha002+6339fce88f07b4d959d03b78330fdb5a6d93